In [1]:
# imports
import pandas as pd

In [2]:
# create column names
features = []
for i in range (1,434):
    feature_name = f"feature_{i}"
    # print(feature_name)
    features.append(feature_name)
# features.append("label")
len(features)

433

In [3]:
# merge test data and labels
def merge(data, labels):
    dataDf = pd.read_csv(data, header=None, names=features, float_precision='round_trip')
    labelsDf = pd.read_csv(labels, header=None, names=['label'])
    # turn labels into ints
    labelsDf['label'] = labelsDf['label'].astype(int)
    merged = pd.merge(dataDf, labelsDf, left_index=True, right_index=True)
    # print(f"data: {dataDf.shape}, labels: {labelsDf.shape}, merged: {merged.shape}")
    # print(f"{data}: \n{dataDf.iloc[4]}")
    return merged


In [ ]:
test_merged = merge("original_data/noExclusion_test_data.csv","original_data/noExclusion_test_label.csv")
train_merged = merge("original_data/noExclusion_train_data.csv","original_data/noExclusion_train_label.csv")    
# print(f"test: {test_merged.shape}, train: {train_merged.shape}")
combined = pd.concat([train_merged,test_merged])
combined['label']
combined.to_csv('combined.csv')
test_merged.to_csv('test_merged.csv')
train_merged.to_csv('train_merged.csv')

In [66]:
import random
random.seed(1)
# to use for sampling 236 NDBE records (iCodeLikeImDrunk, 2012)
indexes = random.sample(range(320), 236)
len(indexes)
# indexes

[68,
 291,
 32,
 130,
 60,
 253,
 230,
 241,
 194,
 107,
 48,
 249,
 14,
 199,
 221,
 1,
 228,
 136,
 117,
 52,
 162,
 15,
 11,
 13,
 277,
 4,
 195,
 110,
 216,
 307,
 270,
 113,
 224,
 314,
 283,
 119,
 176,
 118,
 112,
 235,
 148,
 297,
 213,
 51,
 95,
 151,
 61,
 170,
 256,
 318,
 259,
 97,
 155,
 145,
 255,
 258,
 201,
 17,
 245,
 124,
 206,
 212,
 88,
 187,
 191,
 22,
 281,
 169,
 316,
 27,
 306,
 41,
 133,
 215,
 100,
 94,
 125,
 271,
 7,
 120,
 278,
 78,
 180,
 217,
 157,
 274,
 279,
 261,
 165,
 43,
 313,
 128,
 58,
 3,
 197,
 276,
 138,
 220,
 140,
 59,
 103,
 131,
 257,
 289,
 147,
 90,
 301,
 319,
 168,
 305,
 267,
 186,
 304,
 98,
 200,
 189,
 218,
 33,
 132,
 308,
 143,
 300,
 109,
 290,
 123,
 93,
 266,
 141,
 287,
 129,
 105,
 260,
 91,
 106,
 236,
 0,
 137,
 223,
 159,
 156,
 84,
 295,
 153,
 312,
 227,
 299,
 45,
 210,
 149,
 46,
 23,
 192,
 65,
 8,
 18,
 21,
 294,
 115,
 226,
 71,
 63,
 264,
 28,
 47,
 185,
 74,
 262,
 42,
 40,
 167,
 135,
 229,
 69,
 75,
 116,
 82,
 

In [104]:
# make balanced dataset (balance NDBE and neoplasia)
combined = pd.read_csv('combined.csv', index_col=0).reset_index(drop=True)
combined['label'].value_counts()
# 1: 159, 2: 320, 3: 236
squamous = combined.loc[combined['label']==1].reset_index(drop=True)
ndbe = combined.loc[combined['label']==2].reset_index(drop=True)
neoplasia = combined.loc[combined['label']==3].reset_index(drop=True)

# choose the 236 random records
balanced_ndbe = ndbe.iloc[indexes]
# (LondonRob, 2013)
excluded = ndbe.iloc[~ndbe.index.isin(indexes)]
squamous['label'].value_counts()

# merge back together
balanced = pd.concat([squamous, balanced_ndbe, neoplasia]).reset_index(drop=True)
balanced['label'].value_counts()

2    236
3    236
1    159
Name: label, dtype: int64

In [123]:
# (Hamzah Hafejee, 2022, COMP3611_Coursework_Assessment.ipynb, Comp 3611, University of Leeds)
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit

def stratified_split(df, feature):
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.25,random_state=42)
    
    # returns 2 sets of indexes for test and train
    # hence, .loc is used on the dataset df to retrieve the corresponding records
    for train_index, test_index in split.split(df,df[feature]):
        strat_train_set = df.loc[train_index]
        strat_test_set = df.loc[test_index]
    
    # for set_ in (strat_train_set, strat_test_set):
    #     set_.drop((feature),axis=1,inplace=True)
        
   
    return strat_train_set, strat_test_set

train_set, test_set = stratified_split(balanced, 'label')

training_set=train_set.drop("label",axis=1)
training_labels=train_set["label"].copy()

testing = test_set.drop("label",axis=1)
testing_labels= test_set["label"].copy()

training_set.to_csv('balanced_data/train_data.csv', index=False)
training_labels.to_csv('balanced_data/train_label.csv', index=False)
testing.to_csv('balanced_data/test_data.csv', index=False)
testing_labels.to_csv('balanced_data/test_label.csv', index=False)


202    2
508    3
292    2
375    2
14     1
      ..
488    3
106    1
395    3
295    2
133    1
Name: label, Length: 158, dtype: int64

Manually fixed to make NDBE and neoplasia completely equal